# Text generator based on RNN

### Import libraries

In [1]:
import tensorflow as tf
import numpy as np

### Configurations

In [2]:
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            "\\^_abcdefghijklmnopqrstuvwxyz{|}")
TestText="Too young, too simple, sometimes naive. "
graphPath=r".\graphs"
num_step=len(TestText)
state_size=100#Make it simple
batchSize=1
def Encode(string,name=None):
    tensor=[vocab.find(ch)+1 for ch in string]
    tensor = tf.one_hot(tensor, depth=len(vocab)+1, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32, name=name)
    with tf.Session() as sess:
        nparray=tensor.eval()
    return nparray
def Decode(nparray):
    return "".join([vocab[index-1] if index>0 else "[INVALID]" for index in np.argmax(nparray,axis=1)])
X=Encode(TestText)
Y=X[1:]
X=X[:-1]
print(X.shape,Y.shape)

(39, 84) (39, 84)


### Create graph

In [3]:
with tf.variable_scope("RNN") as modelScope:
    with tf.name_scope("structure"):
        inputs=tf.placeholder(dtype=tf.float32,shape=[None,len(vocab)+1])
        outputs=tf.placeholder(dtype=tf.float32,shape=[None,len(vocab)+1])
        cell=tf.contrib.rnn.GRUCell(num_units=state_size)
        w=tf.get_variable(name="Weight_s_o",shape=[state_size,len(vocab)+1],initializer=tf.truncated_normal_initializer())
        b=tf.get_variable(name="Bias_s_o",shape=[1,len(vocab)+1],initializer=tf.truncated_normal_initializer())
        initState=cell.zero_state(batch_size=batchSize,dtype=tf.float32)
        state=initState
        input_=tf.slice(input_=inputs,begin=[0,0],size=[1,-1])
        testvar=input_
        print(tf.shape(input_))
        netOutputs=[]
        #Since the tf.nn.softmax_cross_entropy_with_logits() runs softmax internally, and 
        #I don't find another function to compute the cross entropy without internal softmax
        #operations, so...
        netVectsForLoss=[]
        for i in range(num_step):
            if(i>0):
                modelScope.reuse_variables()
            output,state=cell(inputs=input_,state=state)
            output=tf.add(tf.matmul(output,w),b)
            netVectsForLoss.append(output)
            output=tf.nn.softmax(output,name="Output")
            netOutputs.append(output)
            input_=output
        netOutputs=tf.concat(values=netOutputs,axis=0)
        netVectsForLoss=tf.concat(values=netVectsForLoss,axis=0,name="vectForLoss")
    with tf.name_scope("training"):
        loss=tf.losses.softmax_cross_entropy(logits=netVectsForLoss,onehot_labels=outputs)
        loss=tf.reduce_mean(loss,name="loss")
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001,name="optimizer").minimize(loss)
    with tf.name_scope("summary") as scope:
        summary_op=tf.summary.merge_all()

Tensor("RNN/structure/Shape:0", shape=(2,), dtype=int32)


ValueError: Variable RNN/RNN/Weight_s_o/optimizer/ does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [ ]:
with tf.Session() as sess:
    writer=tf.summary.FileWriter(logdir=graphPath,graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    result=sess.run(netOutputs,{inputs:X,outputs:Y})
print(Decode(result))